In [1]:
# 导入库
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error  # 评价指标
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,GRU
from keras import optimizers
import keras
 
import tensorflow as tf
 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")#忽略一些警告 不影响运行 
%matplotlib inline
 
 
data=pd.read_csv("600519.csv",encoding='gbk')
data.head()

Using TensorFlow backend.


,日期,股票代码,名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,换手率,成交量,成交金额,总市值,流通市值
0,2021/3/15,'600519,贵州茅台,1975.45,2069.80,1951.15,2050.0,2026.00,-50.55,-2.4951,0.4974,6248487,12459955712,2.480000e+12,2.480000e+12
1,2021/3/12,'600519,贵州茅台,2026.00,2077.00,2002.01,2070.0,2048.00,-22,-1.0742,0.3210,4032251,8181594631,2.550000e+12,2.550000e+12
2,2021/3/11,'600519,贵州茅台,2048.00,2079.99,1961.48,1975.0,1970.01,77.99,3.9589,0.4519,5676897,11521736384,2.570000e+12,2.570000e+12
3,2021/3/10,'600519,贵州茅台,1970.01,1999.87,1967.00,1977.0,1936.99,33.02,1.7047,0.4074,5117174,10136910284,2.470000e+12,2.470000e+12
4,2021/3/9,'600519,贵州茅台,1936.99,2000.00,1900.18,1955.0,1960.00,-23.01,-1.174,0.6549,8226581,16100769579,2.430000e+12,2.430000e+12


In [2]:
data.columns
 
data.drop(['股票代码','名称','日期','涨跌额','涨跌幅','成交金额','总市值','流通市值','成交量','换手率'],axis=1, inplace=True) # 删除操作 
 
data.head()  # 删除无关紧要的特征 
 
data.corr() # 计算相关系数 
 
print(data["收盘价"])

0       1975.45
1       2026.00
2       2048.00
3       1970.01
4       1936.99
         ...   
4737      37.01
4738      37.10
4739      36.38
4740      36.86
4741      35.55
Name: 收盘价, Length: 4742, dtype: float64


In [4]:
data=np.array(data)
print(data[0])
print(data.shape)
 
 
# 数据和标签
train_x=[] # 特征 
train_y=[]  # 标签 
 
a=[]
for i in range(0,4740,10):
    a.append(data[int(i):int(i)+9])
    train_x.append(a)
    a=[]
    train_y.append(data[int(i)+9:int(i)+10][0][0])

[1975.45 2069.8  1951.15 2050.   2026.  ]
(4742, 5)


In [6]:
print(train_y[0:10])
print(len(train_y))
 
print(len(train_x))
print(train_x[0])
 
# 定义归一化函数
def data_guiyihua(data):
    scaler =MinMaxScaler(feature_range=(0, 1))#指定参数的范围 进行归一化 
    data_guiyihua=np.array(data) # 不进行归一化
#     data_guiyihua_2 = data_guiyihua.reshape((data_guiyihua.shape[0], 1,data_guiyihua.shape[1]))
    return data_guiyihua 
 
 
train_x=data_guiyihua(train_x)
train_y=data_guiyihua(train_y)
 
# 划分验证集和测试集
 
x_train,x_test,y_train,y_test = train_test_split(np.array(train_x),np.array(train_y),test_size=0.2)
#                                                                                         0.2 表示  八份训练 2份验证 懂吗？
x_train=x_train.reshape(379,9,5)
y_train=y_train.reshape(379,1)
print(x_train[0])
# x_train = x_train.astype(np.float32)
# y_train =y_train.astype('float64')


x_test=x_test.reshape(95,9, 5)
y_test=y_test.reshape(95,1)
# x_test = x_test.astype('float64')
# y_test =y_test.astype('float64')

print("--------------------------------------")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

[2058.   2456.43 2141.89 2160.9  1873.   1829.   1713.91 1730.05 1674.01
 1699.  ]
474
474
[[[1975.45 2069.8  1951.15 2050.   2026.  ]
  [2026.   2077.   2002.01 2070.   2048.  ]
  [2048.   2079.99 1961.48 1975.   1970.01]
  [1970.01 1999.87 1967.   1977.   1936.99]
  [1936.99 2000.   1900.18 1955.   1960.  ]
  [1960.   2085.   1960.   2074.96 2060.11]
  [2060.11 2095.   1988.   2000.   2033.  ]
  [2033.   2096.   2010.1  2095.   2140.  ]
  [2140.   2149.77 2033.   2040.   2058.  ]]]
[[45.23 45.45 44.35 44.8  44.76]
 [44.76 45.3  44.12 44.2  44.28]
 [44.28 45.4  44.11 45.4  45.62]
 [45.62 46.52 44.9  46.5  46.77]
 [46.77 46.8  45.5  45.5  45.47]
 [45.47 45.5  44.76 44.99 44.92]
 [44.92 44.99 44.13 44.3  44.21]
 [44.21 44.5  43.8  44.2  44.08]
 [44.08 44.2  42.9  42.91 43.23]]
--------------------------------------
(379, 9, 5)
(379, 1)
(95, 9, 5)
(95, 1)


In [8]:
def create_model_1():
  model = keras.models.Sequential([
    keras.layers.LSTM(100,activation='relu', input_shape=(9,5)),
    keras.layers.Dense(32 ,activation='relu'),# 全连接
    keras.layers.Dense(1,activation='relu')# 1个全链接
                      ])  
#   model.compile(loss='binary_crossentropy', optimizer=op) 分类损失函数 和优化器 
    
  model.compile(loss='mean_squared_error', optimizer='adam')  # 回归损失函数和优化器 
  return model
op = optimizers.RMSprop(lr=0.001)
model1 = create_model_1()
model1.summary()
 
 
 
model1.fit(x_train,y_train,validation_data=(x_train, y_train), epochs=200, batch_size=32,verbose=2, shuffle=True)
# model1.save_weights('lstmmoxing')#模型保存
 
# from sklearn.metrics import mean_squared_error #均方误差
# from sklearn.metrics import mean_absolute_error #平方绝对误差
# from sklearn.metrics import r2_score#R square
# #调用
 
# #引用上边的模型实例
# model_jiazai_1 = create_model_1()
# # 加载保存好的模型
# model_jiazai_1.load_weights('lstmmoxing')
# y1_pred_lstm = model_jiazai_1.predict(x_test)
# for i in range(len(y1_pred_lstm )):
#     print("真实收盘价：",y_test[i])
#     print("预测收盘价：",y1_pred_lstm[i])
#     print("-----------------------")
 
# print(mean_squared_error(y_test,y1_pred_lstm))
# print(mean_absolute_error(y_test,y1_pred_lstm))
# print(r2_score(y_test,y1_pred_lstm))
 
 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               42400     
_________________________________________________________________
dense_3 (Dense)              (None, 32)                3232      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 45,665
Trainable params: 45,665
Non-trainable params: 0
_________________________________________________________________
Train on 379 samples, validate on 379 samples
Epoch 1/200
 - 1s - loss: 115216.5814 - val_loss: 7916.4917
Epoch 2/200
 - 0s - loss: 2703.3481 - val_loss: 744.9895
Epoch 3/200
 - 0s - loss: 435.9785 - val_loss: 217.0535
Epoch 4/200
 - 0s - loss: 194.9294 - val_loss: 97.4409
Epoch 5/200
 - 0s - loss: 155.2395 - val_loss: 343.3254
Epoch 6/200
 - 0s - loss: 323.0013 - val_loss: 307

KeyboardInterrupt: 